In [1]:
# import python libraries and spatial packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

# Task 1: Import all datasets (clean if required) into your PostgreSQL server, using a well-defined data schema. T

## Load datasets

In [2]:
# Import Businesses dataset
businesses_rawdf = pd.read_csv('Businesses.csv')
businesses = businesses_rawdf.copy() # prepare a working copy

# Import Stops dataset
stops_rawdf = pd.read_csv('Stops.txt', delimiter=',')
stops_df = stops_rawdf.copy() # prepare a working copy

# Import Polls dataset
polls_rawdf = pd.read_csv('PollingPlaces2019.csv')
polls_df = polls_rawdf.copy() # prepare a working copy


# Import Population dataset
population_rawdf = pd.read_csv('Population.csv')
population_df = population_rawdf.copy() # prepare a working copy

# Import Income dataset
income_rawdf = pd.read_csv('Income.csv')
income_df = income_rawdf.copy() # prepare a working copy

## Load Spatial Data (shapefiles)

### Geopandas Transformations

In [3]:
# Transform polls dataset
polls_df['geom'] = gpd.points_from_xy(polls_df.longitude, polls_df.latitude)  # creating the geometry column
polls_df = polls_df.drop(columns=['latitude', 'longitude', 'the_geom'])  # removing the old latitude/longitude fields

# Transform stops dataset
stops_df['geom'] = gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)  # creating the geometry column
stops_df = stops_df.drop(columns=['stop_lat', 'stop_lon'])  # removing the old latitude/longitude fields

# Load sa2_regions dataset
sa2_regions_path = 'SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp'
sa2_regions_rawgdf = gpd.read_file(sa2_regions_path)

# Load Schools dataset
future_gdf = gpd.read_file('catchments/catchments_future.shp')
primary_gdf = gpd.read_file('catchments/catchments_primary.shp')
secondary_gdf = gpd.read_file('catchments/catchments_secondary.shp')
schools_gdf = gpd.GeoDataFrame(pd.concat([future_gdf, primary_gdf, secondary_gdf], ignore_index=True))

## Database connection

In [4]:
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials copy.json"

# Helper functions provided in tutorial and recommended to be used in Ed

def pgconnect(credentials_filepath, db_schema="public"):
    with open(credentials_filepath) as f:
        db_conn_dict = json.load(f)
        host         = db_conn_dict['host']
        db_user      = db_conn_dict['user']
        db_pw        = db_conn_dict['password']
        default_db   = db_conn_dict['database']
        port         = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db, conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

db, conn = pgconnect(credentials)

Connected successfully.


### SRID Transformations

In [5]:
# Perform SRID transformations

srid = 4326

def create_wkt_element(geom, srid):
    if geom is not None:
        if geom.geom_type == 'Polygon':
            geom = MultiPolygon([geom])
        return WKTElement(geom.wkt, srid)
    else:
        return None

# sa2_Regions Dataset
sa2_regions_gdf = sa2_regions_rawgdf.copy()  # creating a copy of the original for later
sa2_regions_gdf['geom'] = sa2_regions_gdf['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sa2_regions_gdf = sa2_regions_gdf.drop(columns="geometry")  # deleting the old copy

# Stops dataset
stops_df['geom'] = stops_df['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid)) # srid transformation

# Polls dataset
polls_df['geom'] = polls_df['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid)) # srid transformation

# Schools dataset
schools_gdf['geom'] = schools_gdf['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid)) # srid transformation applying function
schools_gdf = schools_gdf.drop(columns="geometry")  # deleting the old copy

## Data Cleaning

In [6]:
# sa2_regions 
sa2_regions_gdf = sa2_regions_gdf[['SA2_CODE21', 'SA2_NAME21', 'SA3_CODE21', 'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'GCC_NAME21', 'AREASQKM21', 'geom']] # filter desired columns
column_mapping = {col: col.lower() for col in sa2_regions_gdf.columns} # Map original column names to lowercase
sa2_regions_gdf = sa2_regions_gdf.rename(columns=column_mapping) # Rename columns with lowercase names
sa2_regions = sa2_regions_gdf.rename(columns={'sa2_code21' : 'sa2_code', 
                                              'sa2_name21' : 'sa2_name', 
                                              'sa3_code21' : 'sa3_code', 
                                              'sa3_name21' : 'sa3_name', 
                                              'sa4_code21' : 'sa4_code', 
                                              'sa4_name21' : 'sa4_name', 
                                              'gcc_name21' : 'gcc_name', 
                                              'areasqkm21' : 'area_sqkm'}) # implement normalized naming conventions
    
# businesses - no cleaning required

# stops
stops = stops_df[['stop_id', 'stop_name', 'parent_station', 'geom']] # select desired columns

# polls
polls = polls_df[['division_id', 'division_name', 'polling_place_id', 'polling_place_name', 'geom']] # filter desired columns


# schools
schools_gdf = schools_gdf[['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'geom']] # filter desired columns
column_mapping = {col: col.lower() for col in schools_gdf.columns} # Map original column names to lowercase
schools_gdf = schools_gdf.rename(columns=column_mapping) # Rename columns with lowercase names
schools = schools_gdf.rename(columns={'use_id' : 'school_id', 
                                      'catch_type' : 'school_type', 
                                      'use_desc' : 'school_name'}) # implement normalized naming conventions
    
# population
population = population_df[['sa2_code', 'sa2_name', '0-4_people', '5-9_people', '10-14_people', '15-19_people', 'total_people']] # filter desired columns (for scoring)

# income
income = income_df.rename(columns={'sa2_code21' : 'sa2_code'}) # normnalize naming conventions

## Data Ingestion

### Define schemas

In [7]:
# Create SA2Regions table
conn.execute(text('''
DROP TABLE IF EXISTS sa2_regions CASCADE;
CREATE TABLE sa2_regions (
    sa2_code VARCHAR(9) PRIMARY KEY,
    sa2_name VARCHAR(255),
    sa3_code VARCHAR(5),
    sa3_name VARCHAR(255),
    sa4_code VARCHAR(3),
    sa4_name VARCHAR(255),
    gcc_name VARCHAR(255),
    area_sqkm NUMERIC,
    geom GEOMETRY(MULTIPOLYGON, 4326),
    CONSTRAINT unique_sa2_code UNIQUE (sa2_code),
    CONSTRAINT unique_sa2_name UNIQUE (sa2_name)
);'''
))

# Create Businesses table
conn.execute(text('''
    DROP TABLE IF EXISTS businesses;
    CREATE TABLE businesses (
    industry_code VARCHAR(2),
    industry_name VARCHAR(255),
    sa2_code VARCHAR(9) REFERENCES sa2_regions(sa2_code),
    sa2_name VARCHAR(255) REFERENCES sa2_regions(sa2_name),
    "0_to_50k_businesses" INTEGER,
    "50k_to_200k_businesses" INTEGER,
    "200k_to_2m_businesses" INTEGER,
    "2m_to_5m_businesses" INTEGER,
    "5m_to_10m_businesses" INTEGER,
    "10m_or_more_businesses" INTEGER,
    total_businesses INTEGER,
    PRIMARY KEY (sa2_code)
);'''
))

# Create Stops table
conn.execute(text('''
DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stop_id VARCHAR(8) PRIMARY KEY,
    stop_name VARCHAR(255),
    parent_station VARCHAR(8),
    geom GEOMETRY(POINT, 4326)
);'''
))

# Create Polls table
conn.execute(text('''
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
    division_id VARCHAR(3),
    division_name VARCHAR(255),
    polling_place_id VARCHAR(10) PRIMARY KEY,
    polling_place_name VARCHAR(255),
    geom GEOMETRY(POINT, 4326)
);'''
))

# Create Schools table
conn.execute(text('''
DROP TABLE IF EXISTS schools;
CREATE TABLE schools (
    school_id VARCHAR(5) PRIMARY KEY,
    school_type VARCHAR(25),
    school_name VARCHAR(255),
    geom GEOMETRY(MULTIPOLYGON, 4326)
);'''
))

# Create Population table
conn.execute(text('''
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code VARCHAR(9) REFERENCES sa2_regions(sa2_code),
    sa2_name VARCHAR(255) REFERENCES sa2_regions(sa2_name),
    "0-4_people" INTEGER, 
    "5-9_people" INTEGER, 
    "10-14_people" INTEGER, 
    "15-19_people" INTEGER, 
    total_people INTEGER,
    PRIMARY KEY (sa2_code)
);'''
))

# Create Income table
conn.execute(text('''
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code VARCHAR(9) REFERENCES sa2_regions(sa2_code),
    sa2_name VARCHAR(255) REFERENCES sa2_regions(sa2_name),
    earners INTEGER,
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER,
    PRIMARY KEY (sa2_code)
);'''
))

conn.commit()

### Populate newly created tables in our database with cleaned data

In [8]:
# Populate newly created tables in our database with cleaned data
try:
    # sa2_regions table
    sa2_regions.to_sql('sa2_regions', con=conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
    print("sa2_regions table populated successfully.")
    #print(query(conn, "select * from sa2_regions"))
except Exception as e:
    print("Error populating sa2_regions table:", e)

try:
    # businesses table
    businesses.to_sql('businesses', con=conn, if_exists='replace', index=False)
    print("businesses table populated successfully.")
    #print(query(conn, "select * from businesses"))
except Exception as e:
    print("Error populating businesses table:", e)

try:
    # stops table
    stops.to_sql('stops', con=conn, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid)})
    print("stops table populated successfully.")
    #print(query(conn, "select * from stops"))
except Exception as e:
    print("Error populating stops table:", e)

try:
    # polls table
    polls.to_sql('polls', con=conn, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid)})
    print("polls table populated successfully.")
    #print(query(conn, "select * from polls"))
except Exception as e:
    print("Error populating polls table:", e)

try:
    # schools table
    schools.to_sql('schools', con=conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
    print("schools table populated successfully.")
    #print(query(conn, "select * from schools"))
except Exception as e:
    print("Error populating schools table:", e)

try:
    # population table
    population.to_sql('population', con=conn, if_exists='replace', index=False)
    print("population table populated successfully.")
    #print(query(conn, "select * from population"))
except Exception as e:
    print("Error populating population table:", e)

try:
    # income table
    income.to_sql('income', con=conn, if_exists='replace', index=False)
    print("income table populated successfully.")
    #print(query(conn, "select * from income"))
except Exception as e:
    print("Error populating income table:", e)

sa2_regions table populated successfully.
businesses table populated successfully.
stops table populated successfully.
polls table populated successfully.
schools table populated successfully.
population table populated successfully.
income table populated successfully.


In [9]:
conn.rollback()

conn.execute(text('''
DROP TABLE IF EXISTS homelessness;
CREATE TABLE homelessness (
    sa2_name_2016 VARCHAR(255) REFERENCES sa2_regions(sa2_name),
    hl_p_homeless_tot INTEGER,
    sa2_main16 VARCHAR(9) REFERENCES sa2_regions(sa2_code),
    PRIMARY KEY (sa2_main16)
);'''
))

homelessness_file = 'Homelessness NSW.json'
gdf = gpd.read_file(homelessness_file)

print(gdf['geometry'])
attributes_to_keep = ['sa2_name_2016', 'hl_p_homeless_tot', 'sa2_main16', 'geometry']
gdf = gdf[attributes_to_keep]
gdf['geometry'] = gdf['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))
table_name = 'homelessness'
gdf.to_sql(table_name, con=conn, if_exists='replace', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid=4326)})

conn.commit()

0      MULTIPOLYGON (((151.49562 -33.21384, 151.49529...
1      MULTIPOLYGON (((151.55449 -33.22373, 151.55446...
2      MULTIPOLYGON (((151.41636 -33.23597, 151.41699...
3      MULTIPOLYGON (((151.41942 -33.26326, 151.42042...
4      MULTIPOLYGON (((151.49651 -33.24227, 151.49830...
                             ...                        
725    MULTIPOLYGON (((151.76003 -32.92809, 151.76044...
726    MULTIPOLYGON (((151.62164 -32.89473, 151.62241...
727    MULTIPOLYGON (((151.76261 -32.90964, 151.76276...
728    MULTIPOLYGON (((151.70159 -32.89717, 151.70121...
729    MULTIPOLYGON (((151.72847 -32.89970, 151.72821...
Name: geometry, Length: 730, dtype: geometry


/var/folders/dc/pss02ks15hj4547twb5z2vlc0000gn/T/ipykernel_53924/838888218.py:19: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))


In [10]:
conn.rollback()

conn.execute(text('''
DROP TABLE IF EXISTS energy;
CREATE TABLE energy (
    sa2_name_2011 VARCHAR(255) REFERENCES sa2_regions(sa2_name),
    sa2_code_2011 VARCHAR(9) REFERENCES sa2_regions(sa2_code),
    year INTEGER,
    land_area_ha INTEGER,
    wu_ag_area_ha INTEGER,
    wu_irrigated_area_ha INTEGER,
    wu_vol_wat_app_ml INTEGER,
    wu_wat_oth_ag_use_ml INTEGER,
    wu_tot_wat_user_ml INTEGER,
    wu_app_rate_ml_per_ha INTEGER,
    esg_gen_meters INTEGER,
    esg_non_gen_meters INTEGER,
    esg_tot_meters INTEGER,
    esg_meter_dwell_gen_elec_pc INTEGER,
    mksfg_net_meters_kwh INTEGER,
    mksfg_gross_meters_kwh INTEGER,
    mksfg_non_gen_meters_kwh INTEGER,
    solar_panel_system INTEGER,
    solar_water_heater INTEGER,
    PRIMARY KEY (sa2_name_2011)
);'''
))

energy_file = 'Energy NSW.json'
gdf = gpd.read_file(energy_file)
attributes_to_keep = [
    'sa2_name_2011',
    'sa2_code_2011',
    'year',
    'land_area_ha',
    'wu_ag_area_ha',
    'wu_irrigated_area_ha',
    'wu_vol_wat_app_ml',
    'wu_wat_oth_ag_use_ml',
    'wu_tot_wat_use_ml',
    'wu_app_rate_ml_per_ha',
    'esg_gen_meters',
    'esg_non_gen_meters',
    'esg_tot_meters',
    'esg_meter_dwell_gen_elec_pc',
    'mksfg_net_meters_kwh',
    'mksfg_gross_meters_kwh',
    'mksfg_non_gen_meters_kwh',
    'solar_panel_system',
    'solar_water_heater',
    'geometry'
]
gdf = gdf[attributes_to_keep]
gdf['geometry'] = gdf['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))
table_name = 'energy'
gdf.to_sql(table_name, con=conn, if_exists='replace', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid=4326)})

conn.commit()

/var/folders/dc/pss02ks15hj4547twb5z2vlc0000gn/T/ipykernel_53924/3743862105.py:54: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))


# Task 2: Compute a score for how ”bustling” each individual neighbourhood

In [11]:
query = '''
CREATE TABLE IF NOT EXISTS z_businesses (
    sa2_code VARCHAR(9) PRIMARY KEY,
    region_score NUMERIC, -- Rename the existing z-score column
    z_score NUMERIC -- New z-score column based on region_score
);

WITH region_scores AS (
    SELECT 
        sa.sa2_code::text AS sa2_code,
        (SUM(b.total_businesses)::float / pop.total_people * 1000) AS region_score
    FROM 
        sa2_regions sa
    LEFT JOIN 
        businesses b ON sa.sa2_code::bigint = b.sa2_code::bigint
    LEFT JOIN 
        population pop ON sa.sa2_code::text = pop.sa2_code::text
    WHERE 
        pop.total_people >= 100
    GROUP BY 
        sa.sa2_code, pop.total_people
)

INSERT INTO z_businesses (sa2_code, region_score)
SELECT 
    sa2_code,
    region_score
FROM 
    region_scores;

WITH stats AS (
    SELECT 
        AVG(region_score) AS mean_region_score,
        STDDEV(region_score) AS stddev_region_score
    FROM 
        z_businesses
)

UPDATE z_businesses
SET z_score = (region_score - mean_region_score) / stddev_region_score
FROM stats;
'''

# Execute the query
conn.execute(text(query))

# Commit the changes
conn.commit()

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "z_businesses_pkey"
DETAIL:  Key (sa2_code)=(121041414) already exists.

[SQL: 
CREATE TABLE IF NOT EXISTS z_businesses (
    sa2_code VARCHAR(9) PRIMARY KEY,
    region_score NUMERIC, -- Rename the existing z-score column
    z_score NUMERIC -- New z-score column based on region_score
);

WITH region_scores AS (
    SELECT 
        sa.sa2_code::text AS sa2_code,
        (SUM(b.total_businesses)::float / pop.total_people * 1000) AS region_score
    FROM 
        sa2_regions sa
    LEFT JOIN 
        businesses b ON sa.sa2_code::bigint = b.sa2_code::bigint
    LEFT JOIN 
        population pop ON sa.sa2_code::text = pop.sa2_code::text
    WHERE 
        pop.total_people >= 100
    GROUP BY 
        sa.sa2_code, pop.total_people
)

INSERT INTO z_businesses (sa2_code, region_score)
SELECT 
    sa2_code,
    region_score
FROM 
    region_scores;

WITH stats AS (
    SELECT 
        AVG(region_s

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS z_stops (
    sa2_code VARCHAR(9) PRIMARY KEY,
    region_score NUMERIC,
    z_score NUMERIC
);

WITH z_scores AS (
    SELECT 
        sa.sa2_code::text AS sa2_code,
        (COUNT(s.stop_id)::float / pop.total_people * 1000) AS region_score
    FROM 
        sa2_regions sa
    LEFT JOIN 
        stops s ON ST_Contains(sa.geom, s.geom)
    LEFT JOIN 
        population pop ON sa.sa2_code::text = pop.sa2_code::text
    WHERE 
        pop.total_people >= 100
    GROUP BY 
        sa.sa2_code, pop.total_people
)

INSERT INTO z_stops (sa2_code, region_score)
SELECT 
    sa2_code,
    region_score
FROM 
    z_scores;

UPDATE z_stops SET z_score = (region_score - avg_score) / stddev_score
FROM (
    SELECT 
        AVG(region_score) AS avg_score,
        STDDEV(region_score) AS stddev_score
    FROM 
        z_stops
) AS stats;
'''

# Execute the query
conn.execute(text(query))

# Commit the changes
conn.commit()

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS z_polls (
    sa2_code VARCHAR(9) PRIMARY KEY,
    region_score NUMERIC,
    z_score NUMERIC
);

WITH z_scores AS (
    SELECT 
        sa.sa2_code::text AS sa2_code,
        (COUNT(p.polling_place_id)::float / pop.total_people * 1000) AS region_score
    FROM 
        sa2_regions sa
    LEFT JOIN 
        polls p ON ST_Contains(sa.geom, p.geom)
    LEFT JOIN 
        population pop ON sa.sa2_code::text = pop.sa2_code::text
    WHERE 
        pop.total_people >= 100
    GROUP BY 
        sa.sa2_code, pop.total_people
)

INSERT INTO z_polls (sa2_code, region_score)
SELECT 
    sa2_code,
    region_score
FROM 
    z_scores;

UPDATE z_polls SET z_score = (region_score - avg_score) / stddev_score
FROM (
    SELECT 
        AVG(region_score) AS avg_score,
        STDEV(region_score) AS stddev_score
    FROM 
        z_polls
) AS stats;
'''

# Execute the query
conn.execute(text(query))

# Commit the changes
conn.commit()

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS z_schools (
    sa2_code VARCHAR(9) PRIMARY KEY,
    region_score NUMERIC,
    z_score NUMERIC
);

WITH z_scores AS (
    SELECT 
        sa.sa2_code::text AS sa2_code,
        (COUNT(sc.school_id)::float / (pop."0-4_people" + pop."5-9_people" + pop."10-14_people" + pop."15-19_people") * 1000) AS region_score
    FROM 
        sa2_regions sa
    LEFT JOIN 
        schools sc ON ST_Intersects(sa.geom, sc.geom)
    LEFT JOIN 
        population pop ON sa.sa2_code::text = pop.sa2_code::text
    WHERE 
        pop.total_people >= 100
    GROUP BY 
        sa.sa2_code, pop."0-4_people", pop."5-9_people", pop."10-14_people", pop."15-19_people"
)

INSERT INTO z_schools (sa2_code, region_score)
SELECT 
    sa2_code,
    region_score
FROM 
    z_scores;

UPDATE z_schools SET z_score = (region_score - avg_score) / stddev_score
FROM (
    SELECT 
        AVG(region_score) AS avg_score,
        STDEV(region_score) AS stddev_score
    FROM 
        z_schools
) AS stats;
'''

# Execute the query
conn.execute(text(query))

# Commit the changes
conn.commit()

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS well_resourced_score (
    sa2_code VARCHAR(9) PRIMARY KEY,
    score NUMERIC
);

INSERT INTO well_resourced_score (sa2_code, score)
SELECT
    z_businesses.sa2_code,
    1 / (1 + EXP(- (z_businesses.z_score + z_stops.z_score + z_polls.z_score + z_schools.z_score))) AS well_resourced_score
FROM
    z_businesses
JOIN
    z_stops ON z_businesses.sa2_code = z_stops.sa2_code
JOIN
    z_polls ON z_businesses.sa2_code = z_polls.sa2_code
JOIN
    z_schools ON z_businesses.sa2_code = z_schools.sa2_code;
'''

# Execute the query
conn.execute(text(query))

# Commit the changes
conn.commit()